In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
def read_YUV_file(file_path, frame_index, frame_height=288, frame_width=352):
    Y_size = frame_height * frame_width
    UV_size = Y_size // 2

    frame_size = Y_size + UV_size

    with open(file_path, 'rb') as f:
        f.seek(frame_size * frame_index)
        Y_data = np.frombuffer(f.read(Y_size), dtype=np.uint8).reshape(frame_height, frame_width)
        U_data = np.frombuffer(f.read(UV_size // 2), dtype=np.uint8).reshape(frame_height // 2, frame_width // 2)
        V_data = np.frombuffer(f.read(UV_size // 2), dtype=np.uint8).reshape(frame_height // 2, frame_width // 2)

    return Y_data

In [6]:
def view_as_blocks(image, block_shape):
    """
    Break the image up into a set of blocks
    Args:
        image: the input image
        block_shape: a tuple specifying the height and width of the blocks

    Returns:
        A 4D numpy array of shape (n_blocks_row, n_blocks_col, block_height, block_width)
    """
    assert len(block_shape) == 2, "Block shape must be a 2-tuple"
    assert len(image.shape) == 2, "Input image must be 2D"
    assert image.shape[0] % block_shape[0] == 0, "Image height is not divisible by block height"
    assert image.shape[1] % block_shape[1] == 0, "Image width is not divisible by block width"

    block_height, block_width = block_shape
    n_blocks_row = image.shape[0] // block_height
    n_blocks_col = image.shape[1] // block_width

    blocks = np.zeros((n_blocks_row, n_blocks_col, block_height, block_width), dtype=image.dtype)

    for i in range(n_blocks_row):
        for j in range(n_blocks_col):
            blocks[i, j] = image[i*block_height:(i+1)*block_height,
                                 j*block_width:(j+1)*block_width]

    return blocks

In [12]:
# Parameters
block_size = 16
search_window_size = 15
# Initialize SAD and computation count
total_sad = 0
total_computations = 0

In [10]:
image1= read_YUV_file('bus_cif.yuv', frame_index=0)
image2 = read_YUV_file('bus_cif.yuv', frame_index=1)

In [11]:
blocks1 = view_as_blocks(image1, block_shape=(block_size, block_size))

In [13]:
# Full search block matching
for i in range(blocks1.shape[0]):
    for j in range(blocks1.shape[1]):
        block1 = blocks1[i, j]

        min_sad = np.inf
        best_match = (0, 0)

        # Define the search window in the second image
        search_window = image2[max(i - search_window_size // 2, 0):min(i + search_window_size // 2, image2.shape[0] - block_size),
                               max(j - search_window_size // 2, 0):min(j + search_window_size // 2, image2.shape[1] - block_size)]

        # Slide the block over the search window
        for m in range(search_window.shape[0] - block_size + 1):
            for n in range(search_window.shape[1] - block_size + 1):
                block2 = search_window[m:m + block_size, n:n + block_size]

                # Compute SAD
                sad = np.sum(np.abs(block1 - block2))
                total_computations += 1

                if sad < min_sad:
                    min_sad = sad
                    best_match = (m, n)

        # Add best SAD to total SAD
        total_sad += min_sad

# Compute average SAD and average number of computations
average_sad = total_sad / (blocks1.shape[0] * blocks1.shape[1])
average_computations = total_computations / (blocks1.shape[0] * blocks1.shape[1])

print(f'Average SAD: {average_sad}')
print(f'Average number of SAD computations: {average_computations}')

Average SAD: inf
Average number of SAD computations: 0.0
